<a href="https://colab.research.google.com/github/silverstar0727/pragmatism-AI/blob/main/ResNet(cifar100).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q wandb

     |████████████████████████████████| 1.8 MB 13.1 MB/s 
     |████████████████████████████████| 170 kB 54.6 MB/s 
     |████████████████████████████████| 133 kB 60.1 MB/s 
     |████████████████████████████████| 97 kB 9.8 MB/s 
     |████████████████████████████████| 138 kB 65.6 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

from wandb.keras import WandbCallback
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
cifar100 = tf.keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=3)

169009152/169001437 [==============================] - 6s 0us/step


In [4]:
def relu_bn(x):
    x = ReLU()(x)
    x = BatchNormalization()(x)
    return x

def residual_block(x, downsample, filters, kernel_size=3):
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(32, 32, 3))
    num_filters = 64
    
    x = BatchNormalization()(inputs)
    x = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(x)
    x = relu_bn(x)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            x = residual_block(x, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
    
    x = AveragePooling2D(4)(x)
    x = Flatten()(x)
    outputs = Dense(100, activation='softmax')(x)
    
    model = Model(inputs, outputs)

    return model

In [5]:
def experiment():
    config_defaults = {
        'epochs': 2,
        'batch_size': 1024,
        'lr': 0.1
    }

    wandb.init(project='ResNet', config=config_defaults, magic=True)
    args = wandb.config
    
    # Prepare dataloader
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    trainloader = trainloader.shuffle(1024).batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    valloader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    valloader = valloader.batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    testloader = testloader.batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    # init
    keras.backend.clear_session()
    
    net = create_res_net()
    optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
    net.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    
    filepath = f'{args.lr}-best.hdf5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                                                    monitor='val_accuracy',
                                                    verbose=1,
                                                    save_best_only=True,
                                                    mode='max')

    result = net.fit(
        trainloader, 
        epochs=args.epochs, 
        validation_data=valloader, 
        batch_size=args.batch_size,
        callbacks=[WandbCallback(save_model=False), checkpoint])

    test_result = net.evaluate(testloader, callbacks=[WandbCallback(save_model=False)])

In [6]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'lr': {
            'values': [0.01, 0.001, 0.0001]
        },
        'epochs': {
            'values': [25]
        },
        'batch_size': {
            'values': [1024]
        }
    }
}

import wandb
sweep_id = wandb.sweep(sweep_config)

wandb.agent(sweep_id, function=experiment)

Create sweep with ID: ndselms6
Sweep URL: https://wandb.ai/jeongmin/uncategorized/sweeps/ndselms6


wandb: Agent Starting Run: 2xnn7cq4 with config:
wandb: 	batch_size: 1024
wandb: 	epochs: 25
wandb: 	lr: 0.01
wandb: Currently logged in as: jeongmin (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: WARNING Ignored wandb.init() arg project when running a sweep


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/25
40/40 [==============================] - 52s 778ms/step - loss: 4.8455 - accuracy: 0.0187 - val_loss: 16608350208.0000 - val_accuracy: 0.0109

Epoch 00001: val_accuracy improved from -inf to 0.01090, saving model to 0.01-best.hdf5
Epoch 2/25
40/40 [==============================] - 26s 652ms/step - loss: 4.3941 - accuracy: 0.0338 - val_loss: 1681099.0000 - val_accuracy: 0.0087

Epoch 00002: val_accuracy did not improve from 0.01090
Epoch 3/25
40/40 [==============================] - 26s 651ms/step - loss: 4.2167 - accuracy: 0.0477 - val_loss: 12450.8779 - val_accuracy: 0.0102

Epoch 00003: val_accuracy did not improve from 0.01090
Epoch 4/25
40/40 [==============================] - 26s 654ms/step - loss: 4.2637 - accuracy: 0.0501 - val_loss: 1322.0063 - val_accuracy: 0.0105

Epoch 00004: val_accuracy did not improve from 0.01090
Epoch 5/25
40/40 [==============================] - 26s 651ms/step - loss: 4.0455 - accuracy: 0.0667 - val_loss: 35.1871 - val_accuracy: 0.0123

Epo

...

wandb: Agent Starting Run: 8ukuh35r with config:
wandb: 	batch_size: 1024
wandb: 	epochs: 25
wandb: 	lr: 0.0001
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/25
40/40 [==============================] - 30s 686ms/step - loss: 4.2917 - accuracy: 0.0564 - val_loss: 4.6416 - val_accuracy: 0.0113

Epoch 00001: val_accuracy improved from -inf to 0.01130, saving model to 0.0001-best.hdf5
Epoch 2/25
40/40 [==============================] - 26s 652ms/step - loss: 3.7888 - accuracy: 0.1355 - val_loss: 4.7502 - val_accuracy: 0.0113

Epoch 00002: val_accuracy did not improve from 0.01130
Epoch 3/25
40/40 [==============================] - 26s 653ms/step - loss: 3.4011 - accuracy: 0.2041 - val_loss: 4.9693 - val_accuracy: 0.0110

Epoch 00003: val_accuracy did not improve from 0.01130
Epoch 4/25
40/40 [==============================] - 26s 653ms/step - loss: 3.0169 - accuracy: 0.2815 - val_loss: 5.3525 - val_accuracy: 0.0141

Epoch 00004: val_accuracy improved from 0.01130 to 0.01410, saving model to 0.0001-best.hdf5
Epoch 5/25
40/40 [==============================] - 26s 653ms/step - loss: 2.6648 - accuracy: 0.3610 - val_loss: 5.0918 - val_accur

epoch,24
loss,0.07572
accuracy,0.989
val_loss,4.55595
val_accuracy,0.2163
_runtime,793
_timestamp,1627196081
_step,24
best_val_loss,4.07871
best_epoch,12


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▇▇▆▅▅▄▄▄▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁
accuracy,▁▂▂▃▃▄▄▄▅▅▆▆▆▇▇▇█████████
val_loss,▂▃▃▄▄▄▆█▃▃▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂
val_accuracy,▁▁▁▁▁▁▂▂▃▄▅▆▇▇▇█▇▇███████
_runtime,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇███
_timestamp,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇███
_step,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


[Experiment](https://wandb.ai/jeongmin/uncategorized/sweeps/ef7doh12?workspace=user-jeongmin)

In [8]:
model = keras.models.load_model('0.001-best.hdf5')

In [77]:
import PIL.Image as pilimg
from keras.preprocessing import image
import numpy as np

img = pilimg.open('airplane.jpeg')
img = img.resize((32, 32))

data = image.img_to_array(img)
data = np.expand_dims(data, axis=0)

def preprocess(data):
	data = data.astype('float32')
	data = data / 255.0
	return data

inputs = preprocess(data)

print(inputs.shape)

(1, 32, 32, 3)


In [78]:
import tensorflow_datasets as tfds

cifar_builder = tfds.builder("cifar100")
cifar_builder.download_and_prepare()
labels = cifar_builder.info.features["label"].names

In [79]:
result = model.predict(inputs)
labels[np.argmax(result)]

'dolphin'

In [80]:
np.max(result)

0.8067352